In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text= open(path_to_file, "rb").read().decode(encoding='utf-8')
print(f'Length of text : {len(text)} characters')

Length of text : 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
example_texts = ['abcdefg', 'xyz']
chars=tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None)
ids=ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token=None, invert=True)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [10]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [11]:
text_from_ids(ids)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'abcdefg', b'xyz'], dtype=object)>

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [15]:
seq_length=100

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print(f'Input Example : {text_from_ids(input_example)}')
    print(f'Target Example : {text_from_ids(target_example)}')

Input Example : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target Example : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [22]:
BATCH_SIZE=64
BUFFER_SIZE=10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [23]:
vocab_size=len(ids_from_chars.get_vocabulary())
embedding_dim=256
rnn_units=1024

In [24]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_state=True, return_sequences=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
    def call(self, inputs, states=None, return_state=False, training=False):
        x=inputs
        x= self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x,states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        if return_state:
            return x, states
        else:
            return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions, states = model(input_example_batch, return_state=True)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 66), dtype=float32, numpy=
array([[-0.00091549,  0.00989263, -0.01140929, ...,  0.01418819,
         0.00300857,  0.0048025 ],
       [ 0.00328853,  0.00044164, -0.01417324, ...,  0.00158249,
        -0.00413412, -0.00527469],
       [-0.00141547,  0.00301661, -0.00258737, ..., -0.00362638,
        -0.01150018,  0.00934668],
       ...,
       [-0.00756153, -0.00038743, -0.00720855, ..., -0.00726603,
         0.01039174, -0.00569758],
       [-0.00862981,  0.0017312 , -0.00678661, ..., -0.00501955,
        -0.00461998, -0.00591494],
       [-0.0045813 , -0.00521695,  0.01091945, ..., -0.00926325,
        -0.0070805 , -0.00677072]], dtype=float32)>

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1)
sampled_indices

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([19,  9,  8,  1, 52, 21, 52, 24, 21, 15, 32, 35, 38, 41, 30, 58, 21,
       44, 19, 12, 58, 45, 11, 21, 55,  6, 54, 17,  7, 61, 58, 19,  8, 65,
       63, 60, 24, 15, 28, 25, 60,  7, 61, 40, 37, 29, 42, 45, 48, 11, 49,
       30, 11,  6, 52, 19, 43, 34,  6, 46, 14, 41, 32, 63, 18,  0, 13,  6,
       31,  3,  5, 30, 53, 40, 12, 19, 54, 61, 37,  5, 29, 59, 17,  0, 57,
        0, 41,  0,  0, 20, 33,  4, 23, 57,  9, 18, 13, 13, 36, 48])>

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ULINA:\nNay, rather, good my lords, be second to me:\nFear you his tyrannous passion more, alas,\nThan '

Next Char Predictions:
 b"F.-\nmHmKHBSVYbQsHeF;sf:Hp'oD,vsF-zxuKBOLu,vaXPcfi:jQ:'mFdU'gAbSxE[UNK]?'R!&Qna;FovX&PtD[UNK]r[UNK]b[UNK][UNK]GT$Jr.E??Wi"


In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1901307, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

66.03142

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS,)

Epoch 1/20
172/172 [==============================] - 122s 701ms/step - loss: 2.6856
Epoch 2/20
172/172 [==============================] - 122s 704ms/step - loss: 1.9667
Epoch 3/20
172/172 [==============================] - 123s 710ms/step - loss: 1.6921
Epoch 4/20
172/172 [==============================] - 122s 705ms/step - loss: 1.5348
Epoch 5/20
172/172 [==============================] - 123s 714ms/step - loss: 1.4394
Epoch 6/20
172/172 [==============================] - 126s 727ms/step - loss: 1.3734
Epoch 7/20
172/172 [==============================] - 123s 711ms/step - loss: 1.3225
Epoch 8/20
172/172 [==============================] - 119s 686ms/step - loss: 1.2773
Epoch 9/20
172/172 [==============================] - 121s 699ms/step - loss: 1.2363
Epoch 10/20
172/172 [==============================] - 121s 698ms/step - loss: 1.1963
Epoch 11/20
172/172 [==============================] - 122s 703ms/step - loss: 1.1572
Epoch 12/20
172/172 [==============================] - 123s 710

In [57]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [58]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [59]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)


result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Then let the feast within me for I'll swear,
Yet art thou not change your cause. Pray you, sir, where go smell.

GRUMIO:
What, myself have spoke her from hencefts being their
Arm use her time?

BRUTUS:
Come, farewell.

PRINCE EDWARD:
Nay, good my lord; and what stir
King Henry he is kindly.'

LUCINTIO:
Were him her mine. If you will bear with palk for hands
The worst is full of poisonous,--

MENENIUS:
They could not dread to harm. I lock his friend,
That may be hep upon your servant; hence fix
themselves lasts.

PETRUCHIO:
Come on, my fortunes to his livery
on thee I may knock. But what a
man is hanged bill'd falsely than an inhorce my daughter
great once, for all the sun that will from thee thee Dike of
That quench my pack fair back-words.

PETRUCHIO:
Pompey?

All:
This, that he die, present the state
To right your valour, with his princely sellcess
So many milthough abidle stones;
And what your highness are the embracements of mine;
Which, though nothing she dreads of steep; y

In [63]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe queen as in a pastly sir, and, for the\nhour all's rock, the duke is my sudge done.\n\nQUEEN MARGARET:\nPoor Clifford, then? who can say he said 'Gains?\n\nFirst Citizen:\nYourself since fair before thy tale.\n\nMURCIUS:\n'Tis good to help met with a virtuous,\nand as I please my hopes, and no looks look intogest\nBy undined browled and up with angel:\nbe singly cloud convey by the year;\nBut now I'll ruin of his their power.\n\nEDWARD:\nSay, each in my made, here will I rele thy death,\nNo fault on evidens in his passage.\n\nLUCIO:\n\nProvost:\nI can better be before thy hand. This I that sin your\ncass.\n\nFirst Senator:\nYou are so; and our fine robes.\n\nLUCENTIO:\nAnd I'll be resolute: then at Paulina's heart I woo'd;\nAnd, paity, give shall in the uteemories,\nLest blood will follow thine enemy: speak, cannot\nplace cuttom, forbear: I make so permit a pox o' the present,\nPrefare the admoning of their prodance,\nOr forbid or round: and here he ceased\nto

In [66]:
print(states.shape)

(5, 1024)
